# Comparison

date: 2018-10-29

bi_ADMM compare with cvxclustr and scvxclustr

compare the running time

compare the results

### install scvxclustr

In [11]:
install.packages('devtools')

Installing package into ‘/Users/yaolanqiu/Library/R/3.4/library’
(as ‘lib’ is unspecified)
also installing the dependencies ‘ps’, ‘processx’, ‘ini’, ‘callr’, ‘desc’, ‘prettyunits’, ‘withr’, ‘xopen’, ‘clisymbols’, ‘fs’, ‘gh’, ‘git2r’, ‘pkgbuild’, ‘pkgload’, ‘rcmdcheck’, ‘remotes’, ‘sessioninfo’, ‘usethis’



In [12]:
library(devtools)

In [13]:
install_github("elong0527/scvxclustr")
library(scvxclustr)

gglasso   (NA      -> 1.4    ) [CRAN]
Matrix    (1.2-12  -> 1.2-14 ) [CRAN]
mclust    (NA      -> 5.4.1  ) [CRAN]
pkgconfig (2.0.1   -> 2.0.2  ) [CRAN]
Rcpp      (0.12.14 -> 0.12.19) [CRAN]


Installing 5 packages: gglasso, Matrix, mclust, pkgconfig, Rcpp
Installing packages into ‘/Users/yaolanqiu/Library/R/3.4/library’
(as ‘lib’ is unspecified)


✔  checking for file ‘/private/var/folders/qy/ldz1ldrs4fl8nm25yvx25z3m0000gn/T/RtmpVFH2HO/remotes7792590b41a7/elong0527-scvxclustr-70da21d/DESCRIPTION’ (340ms)
─  preparing ‘scvxclustr’:
✔  checking DESCRIPTION meta-information
─  cleaning src
─  checking for LF line-endings in source and make files and shell scripts
─  checking for empty or unneeded directories
─  building ‘scvxclustr_0.1.tar.gz’
   


Installing package into ‘/Users/yaolanqiu/Library/R/3.4/library’
(as ‘lib’ is unspecified)
Loading required package: mclust
Package 'mclust' version 5.4.1
Type 'citation("mclust")' for citing this R package in publications.
Loading required package: gglasso

Attaching package: ‘scvxclustr’

The following objects are masked from ‘package:cvxclustr’:

    AMA_step_size, create_adjacency, create_clustering_problem,
    find_clusters



In [2]:
library(scvxclustr)

### install cvxclustr

In [2]:
install.packages('cvxclustr')

Installing package into ‘/Users/yaolanqiu/Library/R/3.4/library’
(as ‘lib’ is unspecified)
also installing the dependency ‘igraph’



In [3]:
library(cvxclustr)

## try the example in scvxclustr package 

In [4]:
simu_4 = function(n, true_p, p, k, mu, sigma, seed = NULL){
  
  if(! is.null(seed) ){ set.seed(seed)  }
  n_p <- true_p / 2
  n_k <- n / k
  clust.ind <- rep(1:k, each = n_k)
  clust.mat <- rbind( c(rep( mu, n_p), rep(-mu, n_p)), 
                      c(rep(-mu, n_p), rep(-mu, n_p)),
                      c(rep(-mu, n_p), rep( mu, n_p)),
                      c(rep( mu, n_p), rep( mu, n_p))
  )
  
  X = matrix(0,n,p)
  for(i in 1:n){
    mu_mean <- c( clust.mat[clust.ind[i],], rep(0, p - true_p) )
    X[i,] <- rnorm(p, mu_mean,rep(sigma,p))
  }
  
  list(X = X, label = clust.ind, features = c(rep(TRUE, true_p), rep(FALSE, p - true_p)))
  
}

In [5]:
n = 400         # Sample size
true_p = 20     # Number of true features
p = 100        # Number of total features (p = 150, 500)
k = 4           # Number of true cluster
mu = 1.2        # mean of normal distribution (mu = 0.6, 0.9)
sigma = 1       # sd of normal distribution
method = "admm"  # Fitted method (method = "ama", "admm")

# Simiulate 4 cluster Gaussian data
data <- simu_4(n = n, true_p = true_p, p = p, k = k, mu = mu, sigma = sigma )

#standardize n by p data matrix
X <- scale(data$X,center=TRUE,scale=FALSE)

In [36]:
# Validation data
data_valide <- list()
for(i in 1:5){
  data_valide[[i]] <- simu_4(n = n, true_p = true_p, p = p, k = k, mu = mu, sigma = sigma )
}

# Adaptive Weight (if possible)
g1 <-6
g2 <- 0
Gamma2.weight <- c(rep(0.5, true_p), rep(1,p - true_p) )
k_w <- 5    # Number of nearest neighbors
phi <- 0.5  # scale of the kernel
verbose <- TRUE # show more information
w <- dist_weight( t(X) / sqrt(p), p = 2 )
w <- knn_weights(w,k = k_w,n)
nu <- AMA_step_size(w,n) /2

In [37]:
nu

[1] 0.02941176

In [18]:
## Validate the cvxclust and scvxclust is the same when g2 = 0
# Fit a convex clustering model
x1 = Sys.time()
fit1 <- cvxclust(X = t(X), w = w, gamma = g1, method = "admm", nu = nu, max_iter = 10, tol = 1e-5)
x2 = Sys.time()
# Fit a sparce convex clsutering model
x3 = Sys.time()
fit2 <- scvxclust(X = X, w = w, Gamma1 = g1, Gamma2 = g2, Gamma2_weight = Gamma2.weight, method = method, nu = nu, max_iter = 10, tol_abs = 1e-5)
x4 = Sys.time()

In [31]:
nu1 = nu2 = nu

In [38]:
n=dim(X)[1]; p=dim(X)[2]
eplison_p=L_num(p)
eplison_n=L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])

v = matrix(1,p,dim(eplison_n)[1])
z = matrix(1,n,dim(eplison_p)[1])

In [44]:
gamma_1= g1; gamma_2 = g2

In [46]:
x5 = Sys.time()
fit3 = bi_ADMM1(X, nu1, nu2, lambda_1, lambda_2, v, z, gamma_1, gamma_2, niter = 10, imax = 0.1)
x6 = Sys.time()

[1] "w"
[1] "s1"
[1] "s2"
[1] "iter"
[1] 1
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 4580383.43535318"
[1] "v 1560079331.71775"
[1] "z 457596641.221375"
[1] "l 885779987.719641"
[1] "2 219780000"
[1] "iter"
[1] 2
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 963408.788932299"
[1] "v 178557565.908679"
[1] "z 51285603.4549954"
[1] "l 29.2461381727227"
[1] "2 9.43969347133589e-11"
[1] "iter"
[1] 3
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 537129.282099573"
[1] "v 127088091.830671"
[1] "z 19575176.633213"
[1] "l 7.32157025994837"
[1] "2 1.19045884261482e-10"
[1] "iter"
[1] 4
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 351735.290622262"
[1] "v 90455359.3439309"
[1] "z 7473303.29039895"
[1] "l 1.4148433059496"
[1] "2 1.39890543593424e-10"
[1] "iter"
[1] 5
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 244073.177945504"
[1] "v 64381572.4435041"
[1] "z 2856169.09431852"
[1] "l 0.674518343058888"
[1] "2 1.58614676948332e-10"
[

In [47]:
x2-x1

Time difference of 4.934234 secs

In [21]:
x4-x3

Time difference of 1.286688 mins

In [48]:
x6-x5

Time difference of 1.140383 mins

In [ ]:
# Compare results

In [8]:
n = 40         # Sample size
true_p = 2     # Number of true features
p = 10       # Number of total features (p = 150, 500)
k = 4           # Number of true cluster
mu = 1.2        # mean of normal distribution (mu = 0.6, 0.9)
sigma = 1       # sd of normal distribution
method = "admm"  # Fitted method (method = "ama", "admm")

# Simiulate 4 cluster Gaussian data
data <- simu_4(n = n, true_p = true_p, p = p, k = k, mu = mu, sigma = sigma )

#standardize n by p data matrix
X <- scale(data$X,center=TRUE,scale=FALSE)

In [9]:
# Validation data
data_valide <- list()
for(i in 1:5){
  data_valide[[i]] <- simu_4(n = n, true_p = true_p, p = p, k = k, mu = mu, sigma = sigma )
}

# Adaptive Weight (if possible)
g1 <-6
g2 <- 0
Gamma2.weight <- c(rep(0.5, true_p), rep(1,p - true_p) )
k_w <- 5    # Number of nearest neighbors
phi <- 0.5  # scale of the kernel
verbose <- TRUE # show more information
w <- dist_weight( t(X) / sqrt(p), p = 2 )
w <- knn_weights(w,k = 1,n)
nu <- AMA_step_size(w,n) /2

In [10]:
n=dim(X)[1]; p=dim(X)[2]
eplison_p=L_num(p)
eplison_n=L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])

v = matrix(1,p,dim(eplison_n)[1])
z = matrix(1,n,dim(eplison_p)[1])

gamma_1= g1; gamma_2 = g2

nu1 = nu2 = nu

In [11]:
dim(X)

[1] 40 10

In [13]:
fit1 <- cvxclust(X = t(X), w = w, gamma = g1, method = "admm", nu = nu, max_iter = 1000, tol = 1e-5)

In [12]:
fit2 <- scvxclust(X = X, w = w, Gamma1 = g1, Gamma2 = g2, Gamma2_weight = Gamma2.weight, method = method, nu = nu, max_iter = 10, tol_abs = 1e-5)

In [14]:
fit3 = bi_ADMM1(X, nu1, nu2, lambda_1, lambda_2, v, z, gamma_1, gamma_2, niter = 1000, imax = 0.1)

[1] "w"
[1] "s1"
[1] "s2"
[1] "iter"
[1] 1
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 44400"
[1] "v 872777.412749703"
[1] "z 202285.714285714"
[1] "l 865771.938418468"
[1] "2 199800"
[1] "iter"
[1] 2
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 216.591023084114"
[1] "v 2356.43243595128"
[1] "z 1347.95148413597"
[1] "l 11.4421800243834"
[1] "2 4.70734562441066e-14"
[1] "iter"
[1] 3
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 160.425778799599"
[1] "v 2132.6808162222"
[1] "z 964.335267453027"
[1] "l 9.39171892840769"
[1] "2 4.9737991503207e-14"
[1] "iter"
[1] 4
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 120.697117776881"
[1] "v 1925.93842773612"
[1] "z 691.154134663988"
[1] "l 9.1442607452478"
[1] "2 5.57331958361829e-14"
[1] "iter"
[1] 5
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 93.5703990809635"
[1] "v 1734.03537676629"
[1] "z 497.085564829078"
[1] "l 8.34158118219468"
[1] "2 6.03961325396085e-14"
[1] "iter"
[1] 6


In [16]:
dim(fit1$U[[1]])

[1] 10 40

In [17]:
dim(fit2$U[[1]])

[1] 40 10

In [19]:
dim(fit3$A)

[1] 40 10

In [25]:
t(fit1$U[[1]]) - fit3$A

-0.54923585,2.0325453,0.46238823,2.06668108,0.40786025,0.31794826,-1.39047440,1.61965923,0.67386519,0.2657558124
-0.64293610,0.9044691,-0.76823110,-0.36643398,-0.62790575,0.03315043,-0.31825846,-1.20348612,0.81463020,0.0265954168
0.66961532,0.5247525,0.02205848,-1.37642902,-0.06484279,0.99241344,0.17158031,-0.14686459,0.23176972,0.0195701909
-1.31086704,0.3552770,-1.38265064,-0.39756801,-0.20602771,-0.29056766,-0.58461296,0.19351156,0.11585522,-0.2348978861
-1.48822503,0.9635764,-0.23863881,-0.04028609,1.24234494,0.72164598,1.06331786,-0.37358531,-0.22397721,-0.3188652133
-0.94541265,0.4420503,-0.34416003,0.95918395,-0.24471581,0.77352325,-0.44150990,0.06947497,0.40061756,-0.0665933889
-2.61662942,1.4404496,-0.81666673,-0.17056528,0.10216761,-0.27520228,-0.71969372,-0.10791893,-0.84725889,-0.8785869083
-0.89161704,0.7766078,-1.92949956,0.28206203,1.67073297,0.66761395,0.46606052,0.28797417,1.14573184,0.4630809042
-2.61199628,1.1364817,2.52195077,0.67764164,0.29860598,0.46484116,0.34704896,-0.70861719,-1.53250831,0.9292873340
-1.07169058,0.4951419,-0.49664293,-0.25478151,-0.73020905,-0.75954306,1.50451446,-2.11036561,-0.59975895,-1.2630411678
0.51941281,0.6947882,-0.17153377,-0.80469229,0.85070915,-0.52202920,-0.45431966,-0.40572932,-1.96151500,-1.6396785961


In [26]:
fit2

$U
$U[[1]]
      [,1] [,2] [,3] [,4] [,5] [,6] [,7] [,8] [,9] [,10]
 [1,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [2,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [3,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [4,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [5,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [6,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [7,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [8,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
 [9,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[10,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[11,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[12,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[13,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[14,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[15,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[16,]  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN
[17,]  NaN  NaN  NaN

In [15]:
fit3$A

0.597985624,-2.24718021,-0.51144010,-2.28500314,-0.45683763,-0.35780047,1.52549437,-1.79141299,-0.74807691,-0.29913981
0.695681427,-1.00695088,0.83563641,0.39629677,0.68115710,-0.04496569,0.33905745,1.31690951,-0.90798523,-0.04019772
-0.736588730,-0.57723791,-0.02426870,1.51408376,0.07132370,-1.09167040,-0.18874445,0.16154863,-0.25495350,-0.02153155
1.437718463,-0.41476114,1.50557408,0.41277431,0.20980896,0.29423381,0.62119189,-0.23687520,-0.14750670,0.22613760
1.627051242,-1.07804466,0.24677366,0.03231569,-1.39003890,-0.81239583,-1.18661653,0.39638110,0.23180520,0.33506965
1.051509229,-0.47424744,0.38799534,-1.04833998,0.27872535,-0.84239609,0.49668518,-0.06949548,-0.42881276,0.08418457
2.870775683,-1.60196523,0.89110694,0.17547438,-0.12465969,0.29143617,0.78369365,0.10873114,0.92136231,0.96019967
0.980788395,-0.85427823,2.12247119,-0.31027218,-1.83782631,-0.73438377,-0.51267266,-0.31677561,-1.26031899,-0.50939505
2.873224336,-1.25014493,-2.77417691,-0.74541550,-0.32847188,-0.51133251,-0.38175972,0.77948527,1.68577506,-1.02222868
1.174878846,-0.55709582,0.54569488,0.27808448,0.80733900,0.83601670,-1.66890963,2.32922672,0.65449465,1.39348640
-0.583318407,-0.77575672,0.17302671,0.88460088,-0.95296173,0.56505849,0.49619697,0.44015098,2.17160610,1.80106101


In [7]:
bi_ADMM1=function(X, nu1, nu2, lambda_1, lambda_2, v, z, gamma_1, gamma_2, niter = 1000, imax = 0.1){
  
  A = X  
    
  print('w')
  e = 0.1
  
  n=dim(X)[1]; p=dim(X)[2]
  
  eplison_p=L_num(p)
  eplison_n=L_num(n)
  print('s1')
  elks = elk(n,p)
  el1 = elks$el1
  el2 = elks$el2
  ek1 = elks$ek1
  ek2 = elks$ek2

  w = dist_weight(X / sqrt(p),phi = 0.5)
  w_l = knn_weights(w,k = 1,n)
  u = dist_weight(t(X) / sqrt(n),phi = 0.5)
  u_k = knn_weights(u,k = 1,p)
  
  print('s2')
    
  A_old = matrix(111,dim(A)[1],dim(A)[2])
  v_old = matrix(111,p,dim(eplison_n)[1])
  z_old = matrix(111,n,dim(eplison_p)[1])
  lambda_1_old = matrix(111,p,dim(eplison_n)[1])
  lambda_2_old = matrix(111,n,dim(eplison_p)[1])
  
  for(iter in 1: niter){
    print('iter')
    print(iter)
    
    # update A
      print('update A')
    A = update_A(X, nu1, nu2, lambda_1, lambda_2, v, z, n, p,
                 el1, el2, ek1, ek2)
    
    al1 = t(A) %*% el1; al2 = t(A) %*% el2
    ak1 = A %*% ek1; ak2 = A %*% ek2
    
    # update vz
      print('update vz')
    vz = update_vz(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1,
                   nu2, n, p, eplison_n, eplison_p, w_l, u_k, 
                   al1, al2, ak1, ak2)
    v = vz[[1]]
    z = vz[[2]]
    
    # update lambda
    print('update lambda')
    lambda = update_lambda(X, A, nu1, nu2,v, z, n, p, 
                           eplison_n, eplison_p, 
                           al1, al2, ak1, ak2)
    
    lambda1 = lambda[[1]]
    lambda2 = lambda[[2]]
    
    print(paste('A',sum(abs(A - A_old))))
    print(paste('v',sum(abs(v - v_old))))
    print(paste('z',sum(abs(z -z_old))))
    print(paste('l',sum(abs(lambda1 - lambda_1_old))))
    print(paste('2',sum(abs(lambda2 - lambda_2_old))))
    
    # whether coverage
    if(sum(abs(A - A_old)) < e & 
       sum(abs(v - v_old)) < e & 
       sum(abs(z - z_old)) < e & 
       sum(abs(lambda1 - lambda_1_old)) < e & 
       sum(abs(lambda2 - lambda_2_old)) < e){
      return(list(A = A, v = v, z = z, lambad_1 = lambda1, lambad_2 = lambda2, niter = iter-1))
      break
    }
    A_old = A; v_old = v; z_old = z; 
    lambda_1_old = lambda1; lambda_2_old = lambda2
  }
  if(iter == niter){
    print(paste('not converge within',iter, 'times'))
    return(list(A = A, v = v, z = z, lambad_1 = lambda1, lambad_2 = lambda2, niter = iter-1))
  }
}

# new function 
library(MASS)
library(Matrix)

sylvester= function(A, B, C, eps = 0.0001){
  
  library(MASS)
  library(Matrix)
  
  A1 = Schur(A)
  Q1 = A1$Q; R1 = A1$T
  
  A2 = Schur(B)
  Q2 = A2$Q; R2 = A2$T 
  C = t(Q1) %*% C %*% Q2
  
  Rsq = R1 * R1
  I = diag(dim(A)[1])
  
  k = 1
  n = dim(R2)[1]
  
  while(k < n + 1){
    if(k < n){
      if(abs(R2[k+1, k]) < eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          temp = (X[,1:(k-1)]) %*% matrix(R2[1:(k-1),k],k-1,1)
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
        # mytry = myTryCatch(solve(left))
        #if(is.null(mytry$error) == 0){er = c(er,tt)}
        k = k+1
      }else{
        r11 = R2[k,k]; r12 = R2[k, k+1]; r21 = R2[k+1, k]; r22 = R2[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        if(k == 1){
          temp2 = temp2
          temp3 = temp3
        }else{
          temps = X[,1:(k-1)] %*% matrix(R2[1:(k-1),k:(k+1)],k-1,2)
          temp2 = temps[,1]
          temp3 = temps[,2]
        }
        b1 = C[,k] - temp2 
        b2 = C[,k+1] - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        X[,k:(k+1)] = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                             (r11*r22 - r12*r21) * I) %*% b_prime 
        k = k+2
      }
    }else{
      if(abs(R2[1, k]) > eps){
        left = R1 + R2[k,k] * I
        right = C[,k]
        temp = matrix(0, dim(X)[1],1)
        if(k == 1){
          temp = temp
        }else{
          temp = (X[,1:(k-1)]) %*% matrix(R2[1:(k-1),k],k-1,1)
        }
        temp = matrix(temp, dim(C)[1],1)
        X[,k] = ginv(left) %*% (right - temp)
        k = k+1
      }else{
        R22 = R2
        R22 = cbind(R2, rep(0,dim(R2)[1]))
        R22 = rbind(R22,rep(0,dim(R2)[1]+1))
        r11 = R22[k,k]; r12 = R22[k, k+1]; r21 = R22[k+1, k]; r22 = R22[k+1, k+1]
        temp2 = matrix(0, dim(X)[1],1);temp3=matrix(0, dim(X)[1],1)
        
        if(k == 1){
          temp2 = temp2
          temp3 = temp3
        }else{
          temps = X[,1:(k-1)] %*% matrix(R22[1:(k-1),k:(k+1)],k-1,2)
          temp2 = temps[,1]
          temp3 = temps[,2]
        }
        
        b1 = C[,k] - temp2 
        b2 = - temp3
        b1_prime = R1 %*% b1 + r22 * b1 - r21 * b2
        b2_prime = R1 %*% b2 + r11 * b2 - r12 * b1
        b_prime = matrix(0, dim(X)[1],2)
        b_prime[,1] = b1_prime; b_prime[,2] = b2_prime
        GOD = ginv(R1 %*% R1 + (r11 + r22) * R1 +
                     (r11*r22 - r12*r21) * I) %*% b_prime 
        X[,k] = GOD[,1]
        k = k+2
      }
    }
  }
  return(Q1 %*% X %*% t(Q2))
}

L_num = function(n){
  t = matrix(0, n*(n-1)/2,2)
  count = 0
  for(i in 1:(n-1)){
    for(j in (i+1):n){
      count = count + 1
      t[count,1]= i; t[count,2]=j
    }
  }
  LL = data.frame(l1 = t[,1],l2 = t[,2])
  LL = LL[order(LL$l2),]
  rownames(LL) = NULL
  return(LL)
}

alk = function(A,n,p){
  count = 0
  el1 = el2 = matrix(0,n,n*(n-1)/2)
  for(i in 1:(n-1)){
    el1[,(count+1):(count+i)] = diag(1,n,i)
    temp = matrix(0,n,i)
    temp[i,] = 1
    el2[,(count+1):(count+i)] = temp
    count = count+i
  }
  el2 = rbind(el2[n,],el2[1:(n-1),])
  al1 = t(A) %*% el1; al2 = t(A) %*% el2
  
  # k
  count = 0
  ek1 = ek2 = matrix(0,p,p*(p-1)/2)
  for(i in 1:(p-1)){
    ek1[,(count+1):(count+i)] = diag(1,p,i)
    temp = matrix(0,p,i)
    temp[i+1,] = 1
    ek2[,(count+1):(count+i)] = temp
    count = count+i
  }
  ak1 = A %*% ek1; ak2 = A %*% ek2
  return(list(al1 = al1, al2 = al2, ak1 = ak1, ak2 = ak2,
              el1 = el1, el2 = el2, ek1 = ek1, ek2 = ek2))
}

elk = function(n,p){
  count = 0
  el1 = el2 = matrix(0,n,n*(n-1)/2)
  for(i in 1:(n-1)){
    el1[,(count+1):(count+i)] = diag(1,n,i)
    temp = matrix(0,n,i)
    temp[i,] = 1
    el2[,(count+1):(count+i)] = temp
    count = count+i
  }
  el2 = rbind(el2[n,],el2[1:(n-1),])
  
  # k
  count = 0
  ek1 = ek2 = matrix(0,p,p*(p-1)/2)
  for(i in 1:(p-1)){
    ek1[,(count+1):(count+i)] = diag(1,p,i)
    temp = matrix(0,p,i)
    temp[i+1,] = 1
    ek2[,(count+1):(count+i)] = temp
    count = count+i
  }
  return(list(el1 = el1, el2 = el2, ek1 = ek1, ek2 = ek2))
}


dist_weight <- function(X, phi){
  dist_X <- as.numeric(dist(X))
  return(exp(- phi * dist_X^2))
}

knn_weights <- function(w,k,n) {
  i <- 1
  neighbors <- tri2vec(i,(i+1):n,n)
  keep <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
  for (i in 2:(n-1)) {
    group_A <- tri2vec(i,(i+1):n,n)
    group_B <- tri2vec(1:(i-1),i,n)
    neighbors <- c(group_A,group_B)
    knn <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
    keep <- union(knn,keep)
  }
  i <- n
  neighbors <- tri2vec(1:(i-1),i,n)
  knn <- neighbors[sort(w[neighbors],decreasing=TRUE,index.return=TRUE)$ix[1:k]]
  keep <- union(knn,keep)
  w[-keep] <- 0
  return(w)
}

tri2vec <- function(i,j,n) {
  return(n*(i-1) - i*(i-1)/2 + j -i)
}

prox = function(v,sigma){
  return(max(1-sigma/sum(t(v) %*% v),0) * v)
}

update_A = function(X, nu1, nu2, lambda_1, lambda_2, v, z,n,p,el1,el2,ek1,ek2){
  
  En = diag(0:(n - 1)) + diag((n - 1):0) - matrix(1, n, n) + diag(1, n, n)
  Ep = diag(0:(p - 1)) + diag((p - 1):0) - matrix(1, p, p) + diag(1, p, p)
  
  M = diag(1,n,n) + nu1 * En
  
  N = nu2 * Ep
  
  lv = lambda_1+ nu1 * v
  lz = lambda_2 + nu2 * z
  C2 = 0 -(el2-el1) %*% t(lv)
  C3 = lz %*% t(ek1-ek2)
  C = X +  C2 + C3  
  
  A = sylvester(M,t(N),C)
  return(A)
}

update_vz = function(X, A, lambda_1, lambda_2, gamma_1, gamma_2, nu1,
                     nu2, n, p, eplison_n, eplison_p, w_l, u_k, 
                     al1, al2, ak1, ak2){
  
  sigma_1 = gamma_1 * w_l/nu1
  vtemp = al1 - al2 - 1/nu1 * lambda_1
  
  temp1 = ifelse((1 - sigma_1/apply(vtemp^2,2,sum)) < 0, 0,1 - sigma_1/apply(vtemp^2,2,sum))
  temp2 = matrix(temp1,dim(vtemp)[1],dim(vtemp)[2], byrow = TRUE) * vtemp 
  
  v = temp2

  ztemp = ak1 - ak2 - 1/nu2 * lambda_2
  sigma_2 = gamma_2 * u_k/nu2
  
  temp3 = ifelse((1 - sigma_2/apply(ztemp^2,2,sum)) < 0, 0,1 - sigma_2/apply(ztemp^2,2,sum))
  temp4 = matrix(temp3,dim(ztemp)[1],dim(ztemp)[2], byrow = TRUE) * ztemp 
  
  z = temp4
  return(list(v=v,z=z))
}

update_lambda = function(X, A, nu1, nu2,v, z, n, p, 
                       eplison_n, eplison_p, 
                       al1, al2, ak1, ak2){
  
  # update lambda 1
  lambda_1 = lambda_1 + nu1 * (v - al1 + al2)
  
  # update lambda 2
  lambda_2 = lambda_2 + nu2 * (z - ak1 + ak2)
  return(lambda=list(lambda_1,lambda_2))
}

In [27]:
# prepare data
X = matrix(1:12,4,3)
A = X

nu1=1; nu2=1; gamma_1 = 0.5; gamma_2 = 0

n=dim(X)[1]; p=dim(X)[2]
eplison_p=L_num(p)
eplison_n=L_num(n)

lambda_1 = matrix(1,p,dim(eplison_n)[1])
lambda_2 = matrix(1,n,dim(eplison_p)[1])

v = matrix(1,p,dim(eplison_n)[1])
z = matrix(1,n,dim(eplison_p)[1])

In [28]:
bi_ADMM1(X, nu1, nu2, lambda_1, lambda_2, v, z, gamma_1, gamma_2, niter = 1000, imax = 0.1)

[1] "w"
[1] "s1"
[1] "s2"
[1] "iter"
[1] 1
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 1254"
[1] "v 1996.86275501304"
[1] "z 1344"
[1] "l 1996.86275501304"
[1] "2 1332"
[1] "iter"
[1] 2
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 9.33054422164272"
[1] "v 9.31569572037907"
[1] "z 16"
[1] "l 1.2822263800917"
[1] "2 5.32907051820075e-15"
[1] "iter"
[1] 3
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 6.97010884432855"
[1] "v 7.09112838753077"
[1] "z 12"
[1] "l 0.271323161504955"
[1] "2 3.5527136788005e-15"
[1] "iter"
[1] 4
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 5.26809635558012"
[1] "v 5.84117105452358"
[1] "z 9.00000000000001"
[1] "l 0.276797609100949"
[1] "2 0"
[1] "iter"
[1] 5
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 4.01071809664423"
[1] "v 4.82774421213198"
[1] "z 6.75"
[1] "l 0.138080339072676"
[1] "2 0"
[1] "iter"
[1] 6
[1] "update A"
[1] "update vz"
[1] "update lambda"
[1] "A 3.04774304042224"
[1] "v 3.9377

$A
         [,1]     [,2]      [,3]
[1,] 1.103995 5.098643  9.093292
[2,] 1.953375 5.948024  9.942672
[3,] 3.057328 7.051976 11.046625
[4,] 3.906708 7.901357 11.896005

$v
         [,1]      [,2]      [,3]     [,4]      [,5]     [,6]
[1,] -1.79472 -2.953333 -2.103952 -3.77613 -2.953333 -1.79472
[2,] -1.79472 -2.953333 -2.103952 -3.77613 -2.953333 -1.79472
[3,] -1.79472 -2.953333 -2.103952 -3.77613 -2.953333 -1.79472

$z
          [,1]      [,2]      [,3]
[1,] -4.994649 -8.989297 -4.994649
[2,] -4.994649 -8.989297 -4.994649
[3,] -4.994649 -8.989297 -4.994649
[4,] -4.994649 -8.989297 -4.994649

$lambad_1
          [,1]          [,2]          [,3]       [,4]          [,5]      [,6]
[1,] 0.0546607  0.000000e+00  0.000000e+00 0.02658324 -4.440892e-16 0.0546607
[2,] 0.0546607 -8.881784e-16 -8.881784e-16 0.02658324  0.000000e+00 0.0546607
[3,] 0.0546607  0.000000e+00  0.000000e+00 0.02658324  0.000000e+00 0.0546607

$lambad_2
              [,1]          [,2] [,3]
[1,]  0.000000e+00  0.000000e+00    0
[2,]  0.000000e+00 -1.776357e-15    0
[3,] -8.881784e-16  0.000000e+00    0
[4,]  0.000000e+00  0.000000e+00    0

$niter
[1] 23